In [1]:
import geopandas as gpd
import pandas as pd
import geojson
import numpy as np

In [2]:
# Se carga el dataframe generado por el módulo "Mapa_RC_SZ"
df = gpd.read_file('dataframe.geojson')

# Se carga el geojson de los mapas de bogota por upz y cundinamarca por municipio
with open('bocu.geojson') as f:
    js = geojson.load(f)

# A cada SZ y RC se le adiciona una columna llamada 'ubicacion' donde se identifica la upz o municipio al
# cual corresponde según sus coordenadas 

from shapely.geometry import shape, Point

df['ubicacion'] = "otra"

for feature in js['features']:
    polygon = shape(feature['geometry'])
    for i in range(0, len(df)):
        if polygon.contains(df.loc[i, 'geometry']):
            df.at[i,'ubicacion'] = feature["properties"]["nombre"]     

In [3]:
# Se genera la base 'ubi' donde se consignan la cantidad de equipos con disponibilidad buena, aceptable,
# mala y NaN (sin informacion)

bocu = gpd.read_file('bocu.geojson')
ubi = pd.DataFrame({'upz_o_municipio': ['x'],'dis_buena': [0], 'dis_aceptable': [0], 'dis_mala': [0], 'dis_NaN': [0],
                    'fa_buena': [0], 'fa_aceptable': [0], 'fa_mala': [0], 'fa_NaN': [0]})

for upz_municipio, i in zip(bocu['nombre'], range(0, len(bocu)) ):
    indi = (df[df['ubicacion'].str.contains(upz_municipio)])['color_indi'].value_counts()
    fall = (df[df['ubicacion'].str.contains(upz_municipio)])['color_fallas'].value_counts()
    ubi.at[i, 'upz_o_municipio'] = upz_municipio
    
    try:
        ubi.at[i, 'dis_buena'] = indi['green']
    except:
        ubi.at[i, 'dis_buena'] = 0
    try:
        ubi.at[i, 'dis_aceptable'] =  indi['gold']
    except: 
        ubi.at[i, 'dis_aceptable'] =  0
    try:    
        ubi.at[i, 'dis_mala'] = indi['red']
    except:     
        ubi.at[i, 'dis_mala'] = 0
    try:
        ubi.at[i, 'dis_NaN'] = indi['black']
    except:
        ubi.at[i, 'dis_NaN'] = 0
        
    try:
        ubi.at[i, 'fa_buena'] = fall['green']
    except:
        ubi.at[i, 'fa_buena'] = 0
    try:
        ubi.at[i, 'fa_aceptable'] =  fall['gold']
    except: 
        ubi.at[i, 'fa_aceptable'] =  0
    try:    
        ubi.at[i, 'fa_mala'] = fall['red']
    except:     
        ubi.at[i, 'fa_mala'] = 0
    try:
        ubi.at[i, 'fa_NaN'] = fall['black']
    except:
        ubi.at[i, 'fa_NaN'] = 0

In [4]:
# Se generan las columnas 'Hi_indi' y 'Hi_falla' donde se consigna el HI del municipio o upz según los datos
# de fallas e indisponibilidades de RC y SZ que albergan

for i in range(0, len(ubi)): 
    if (ubi.loc[i,'dis_buena'] + ubi.loc[i,'dis_aceptable'] + ubi.loc[i,'dis_mala']) >= ubi.loc[i,'dis_NaN']:
        ubi.at[i, 'Hi_indi'] = (ubi.loc[i, 'dis_buena'] + 
                            ubi.loc[i, 'dis_aceptable']*0.5) / (ubi.loc[i, 'dis_buena'] + 
                            ubi.loc[i, 'dis_aceptable'] + 
                            ubi.loc[i, 'dis_mala'])
    else:
        ubi.at[i, 'Hi_indi'] = None
        
        
    ubi.at[i, 'Hi_falla'] = (ubi.loc[i, 'fa_buena'] + ubi.loc[i, 'fa_NaN'] +
                        ubi.loc[i, 'fa_aceptable']*0.5) / (ubi.loc[i, 'fa_buena'] + 
                        ubi.loc[i, 'fa_aceptable'] + ubi.loc[i, 'fa_NaN'] +
                        ubi.loc[i, 'fa_mala'])
    
# Nota: al correr esta línea, se lanzan dos RuntimeWarning que no tiene mayor importancia

c:\users\daniel\appdata\local\programs\thonny\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
c:\users\daniel\appdata\local\programs\thonny\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


In [5]:
# Ahora a la base con los RC y SZ 'df' se le adiciona el HI según la informacion histórica de indisponibilidades y fallas
# de acuerdo con la upz o municipio al que pertenece, se llama 'Hi_histo_ubi'
# Ojo: Recordar que ese HI está asociada a la upz o municipio, no al equipo

for i in range(0, len(df)):
    for j in range(0, len(ubi)):
        if  df.loc[i, 'ubicacion'] == ubi.loc[j, 'upz_o_municipio']:
            df.at[i, 'Hi_histo_ubi'] = (ubi.loc[j, 'Hi_indi'] + ubi.loc[j, 'Hi_falla'])*50

In [6]:
file_name = 'prueba_hist.xlsx'
  
# saving the excel
df.to_excel(file_name)

In [6]:
df['DIRECCION']

,COOR_X,COOR_Y,DIRECCION,Disp_sin_0,Disponibilidad,EQUIPO,MARCA,MODELO,Numero_Fallas,TIPO,color_fallas,color_indi,id,geometry,ubicacion,Hi_histo_ubi
0,-74.446528,3.964778,VIA PEÃ‘AS BLANCAS CABRERA,94.000000,94.000000,SZ3888,Col Giovanni Paolo,UP2008-UE8,3.0,SZ,gold,gold,0,POINT (-74.44653 3.96478),Cabrera,83.585859
1,-74.478164,3.977693,SR LA ESCUELA VE JORDAN MUN FUSAGASUGA,NaN,0.000000,RC1403,SCHNEIDER,NULEC ADVC3,3.0,RC,gold,black,1,POINT (-74.47816 3.97769),Cabrera,83.585859
2,-74.485691,3.981824,VIA CABRERA VENECIA MUN DE CABRERA,100.000000,98.000000,SZ4220,Col Giovanni Paolo,UP2008-UE8,3.0,SZ,gold,green,2,POINT (-74.48569 3.98182),Cabrera,83.585859
3,-74.484142,3.983067,CRA 2A - CLL 2,99.000000,99.000000,SZ7137,Advanced Devices,UP2015-UE8,2.0,SZ,green,green,3,POINT (-74.48414 3.98307),Cabrera,83.585859
4,-74.486091,3.983380,VEREDA SAN ISIDRO,99.000000,99.000000,SZ7139,Col Giovanni Paolo,UP2015-UE8,NaN,SZ,black,green,4,POINT (-74.48609 3.98338),Cabrera,83.585859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7977,-74.410864,5.756162,"VEREDA PATEVACA, CERCA A LA ESCUELA",96.000000,53.000000,SZ4644,DUCATI,UP2008-UE8,1.0,SZ,green,green,7979,POINT (-74.41086 5.75616),Yacopí,61.166667
7978,-74.415740,5.756709,SALIDA PATEVACA VIA EL CASTILLO A LADO DERECHO...,91.013462,19.317143,RC2284,NOJA POWER,NOJA RC10,13.0,RC,red,gold,7980,POINT (-74.41574 5.75671),Yacopí,61.166667
7979,-74.632396,5.760391,Â VIA PTO LIBRE PTO BOYACA PASANDO PUENTE RIO ...,NaN,0.000000,ERC5134,NOJA POWER,NOJA RC10,3.0,RC,gold,black,7981,POINT (-74.63240 5.76039),otra,NaN
7980,-73.582603,5.941104,None,NaN,NaN,RC2184,NOJA POWER,None,NaN,RC,black,black,7982,POINT (-73.58260 5.94110),otra,NaN


In [7]:
# Rangos: Buena 100 - 71, Aceptable 70 - 36, Mala 35 - 0
# Indisponibilidad: Buena 100 - 96, Aceptable 95 - 91, 90 - 0
# Fallas: Buena 0 - 2, Aceptable 3 - 5, Mala 6 - 10
# Nota.... El rango de indisponibilidades está con signo negativo por limitantes de np.interp, sin embargo,
#          también se cambia el signo del numero a interpolar para compensar

rangos = [100, 71, 35, 0]
rangos_fallas = [0, 2, 5, 10]
rangos_indisponibilidades = [-100, -96, -91, -0]

# Se calcula el HI según las fallas e indisponibilidades
for i in range(0 ,len(df)):
    df.at[i ,'Hi_indi'] = np.interp(-df.loc[i ,'Disp_sin_0'] , rangos_indisponibilidades, rangos)
    df.at[i ,'Hi_fallas'] = np.interp(df.loc[i ,'Numero_Fallas'] , rangos_fallas, rangos)  

In [8]:
df.to_excel('HI_indisponibilidad_fallas_historico.xlsx')

In [9]:
#Para generar el mapa de HI segun los historicos de fallas e indisponibilidades
import folium

bocu['Hi_histo_ubi'] = (ubi['Hi_indi'] + ubi['Hi_falla'])*50

m = folium.Map(location=[4.7, -74], tiles="Cartodb Positron", zoom_start=8)
#linear = cm.LinearColormap(["green", "yellow", "red"], vmin=3, vmax=10)


folium.Choropleth(
    bocu, data=bocu,
    columns=["nombre", "Hi_histo_ubi"],
    key_on="feature.properties.nombre",
    fill_color="RdYlGn",
    legend_name="Índice de Salud (HI) por zona",
     ).add_to(m)

folium.GeoJson(
    bocu, 
    popup = folium.GeoJsonPopup(fields=["nombre","Hi_histo_ubi"], aliases=["Nombre", "HI por ubicación"] ),
    style_function=lambda feature: {"weight": 0,"fillOpacity": 0},
    tooltip = folium.GeoJsonTooltip(fields=["nombre"])
                  ).add_to(m)

m.save('mapa_HI_historico.html')